In [1]:
print("ok")

ok


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\Ayush Singh\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')


In [5]:
# Extract data from pdf
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents


In [6]:
extracted_data = load_pdf("Data/")

In [7]:
extracted_data[3].page_content

'STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow, Manager, Imaging and Multimedia\nContent\nRobyn V . Young, Project Manager, Imaging and\nMultimedia Content\nDean Dauphinais, Senior Editor, Imaging and\nMultimedia Content\nKelly A. Quin, Editor, Imaging and Multimedia Content\nLeitha Etheridge-Sims, Mary K. Grimes, Dave Oblender,\nImage Catalogers\nPamela A. Reed, Imaging Coordinator\nRandy Bassett, Imaging Supervisor\nRobert Duncan, Senior Imaging Specialist\nDan Newell, Imaging Specialist\nChristine O’Bryan, Graphic Specialist\nMaria Franklin, Permissions Manager\nMargaret A. Chamberlain, Permissions Specialist\nMichelle DiMercurio, Senior Art Director\nMike Logusz, Graphic Artist\n

In [8]:
# Create Text Chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap =20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("Length of text chunks: ", len(text_chunks))

Length of text chunks:  7020


In [10]:
text_chunks[3].page_content

'Multimedia Content\nKelly A. Quin, Editor, Imaging and Multimedia Content\nLeitha Etheridge-Sims, Mary K. Grimes, Dave Oblender,\nImage Catalogers\nPamela A. Reed, Imaging Coordinator\nRandy Bassett, Imaging Supervisor\nRobert Duncan, Senior Imaging Specialist\nDan Newell, Imaging Specialist\nChristine O’Bryan, Graphic Specialist\nMaria Franklin, Permissions Manager\nMargaret A. Chamberlain, Permissions Specialist\nMichelle DiMercurio, Senior Art Director\nMike Logusz, Graphic Artist'

In [4]:
# Download Embedding Model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [5]:
embeddings = download_hugging_face_embeddings()

In [6]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [7]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [15]:
query_result

[-0.034477222710847855,
 0.031023189425468445,
 0.00673493230715394,
 0.02610897831618786,
 -0.03936203941702843,
 -0.16030248999595642,
 0.06692398339509964,
 -0.006441492587327957,
 -0.04745054617524147,
 0.01475894171744585,
 0.07087540626525879,
 0.05552757903933525,
 0.019193286076188087,
 -0.026251375675201416,
 -0.010109512135386467,
 -0.026940539479255676,
 0.022307513281702995,
 -0.022226620465517044,
 -0.14969266951084137,
 -0.017493048682808876,
 0.00767626753076911,
 0.0543522946536541,
 0.003254437353461981,
 0.03172598034143448,
 -0.0846213847398758,
 -0.029406018555164337,
 0.05159567669034004,
 0.048124030232429504,
 -0.0033148014917969704,
 -0.05827920511364937,
 0.04196932911872864,
 0.022210698574781418,
 0.12818889319896698,
 -0.022338908165693283,
 -0.011656295508146286,
 0.06292832642793655,
 -0.03287626430392265,
 -0.09122607111930847,
 -0.03117542713880539,
 0.052699606865644455,
 0.0470348484814167,
 -0.08420299738645554,
 -0.030056146904826164,
 -0.02074474096

In [7]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("mchatbot")

In [8]:
index

In [21]:
for i, t in zip(range(len(text_chunks)), text_chunks):
   query_result = embeddings.embed_query(t.page_content)
   index.upsert(
   vectors=[
        {
            "id": str(i),  # Convert i to a string
            "values": query_result, 
            "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
        }
    ],
    namespace="real" 
)

index.describe_index_stats() 


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'real': {'vector_count': 7101}},
 'total_vector_count': 7101}

In [10]:
docsearch = Pinecone.from_existing_index(index, embeddings)

query = "What are Allergies"

docs = docsearch.similarity_search(query, k = 3)

print("Result", docs)

AttributeError: type object 'Pinecone' has no attribute 'from_existing_index'

In [9]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'real': {'vector_count': 7020}},
 'total_vector_count': 7020}

In [10]:
query = "What are Homeopathy"
mbed_query = embeddings.embed_query(query)
response = index.query(vector = mbed_query, top_k=3,namespace='real', include_metadata=True)

In [11]:
response

{'matches': [{'id': '3700',
              'metadata': {'text': 'Homeopathy and traditional Chinese '
                                   'medicine\n'
                                   'approach anxiety as a symptom of a '
                                   'systemic disorder.Homeopathic '
                                   'practitioners select a remedy based on '
                                   'otherassociated symptoms and the patient’s '
                                   'general constitution.Chinese medicine '
                                   'regards anxiety as a blockage of qi,o r\n'
                                   'vital force, inside the patient’s body '
                                   'that is most likely toaffect the lung and '
                                   'large intestine meridian flow. The '
                                   'practi-tioner of Chinese medicine chooses '
                                   'acupuncture point'},
              'score': 0.48241

In [20]:
mbed_query

[0.018147820606827736,
 -0.007762205321341753,
 0.016550712287425995,
 -0.02258739434182644,
 0.08301928639411926,
 0.04213036969304085,
 0.047391317784786224,
 0.009959197603166103,
 -0.03739217668771744,
 0.07155653089284897,
 0.03732003644108772,
 -0.06228748708963394,
 0.017921388149261475,
 0.01487040426582098,
 -0.03633667901158333,
 0.08066536486148834,
 -0.03342956304550171,
 -0.045790895819664,
 -0.0021369371097534895,
 -0.05767640471458435,
 -0.014061146415770054,
 0.0995464026927948,
 -0.008254234679043293,
 -0.005863514728844166,
 -0.05176495015621185,
 0.06957107037305832,
 -0.037494104355573654,
 -0.09890788793563843,
 -0.05588377267122269,
 -0.03587561100721359,
 -0.04290856420993805,
 -0.023026956245303154,
 0.002322728745639324,
 0.0567626915872097,
 -0.04027232155203819,
 0.0024839863181114197,
 -0.08057363331317902,
 0.02013392373919487,
 0.02193737030029297,
 0.007606549188494682,
 -0.026822276413440704,
 0.023933090269565582,
 -0.06894038617610931,
 -0.037084188312

In [100]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index_name = 'mchatbot'
index = pc.Index(index_name)

docsearch = Pinecone(
    index = index, embedding_function=embeddings.embed_query, text_key = text_field
)

In [12]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore(index_name='mchatbot', embedding=embeddings, namespace='real',
                                pinecone_api_key = PINECONE_API_KEY)

In [13]:
query = "What are Allergies?"

docsearch.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)



[Document(page_content='the itchy, scratchy nose, eyes, and throat common inallergic rhinitis .\nThe particular allergens to which a person is sensi-'),
 Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"),
 Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnu

In [14]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [15]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [16]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-6627fca5-26ae0b6002767d744d44b238;45b950f1-1c5d-493a-b8b3-acb86a742fe8)

Repository Not Found for url: https://huggingface.co/api/models/model/llama-2-7b-chat.ggmlv3.q4_0.bin/revision/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.

In [19]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract methods _aget_relevant_documents, _get_relevant_documents (type=type_error)

In [85]:
while True:
    user_input = input(f"Input Prompt:")
    result = qa({"query": user_input})
    print("Response: ", result['result'])

ValueError: The argument order for `query()` has changed; please use keyword arguments instead of positional arguments. Example: index.query(vector=[0.1, 0.2, 0.3], top_k=10, namespace='my_namespace')

In [109]:
# BY me modifications

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k':2})
)

ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract methods _aget_relevant_documents, _get_relevant_documents (type=type_error)

In [68]:
qa.run("What are Acne")

ValueError: The argument order for `query()` has changed; please use keyword arguments instead of positional arguments. Example: index.query(vector=[0.1, 0.2, 0.3], top_k=10, namespace='my_namespace')

In [46]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)

In [47]:
qa_with_sources(query)

ValueError: The argument order for `query()` has changed; please use keyword arguments instead of positional arguments. Example: index.query(vector=[0.1, 0.2, 0.3], top_k=10, namespace='my_namespace')